# HDF5 export: Compare the size between double/simple precision

#### 1st step : load dpf

In [1]:
from ansys.dpf import core as dpf
import os

In [2]:
path =  r'c:\temp\file_transient.rst'
model = dpf.Model(path)

E0114 22:06:49.777000000  5328 server_chttp2.cc:40] {"created":"@1610658409.777000000","description":"No address added out of total 1 resolved","file":"C:\.conan\9877f9d9b72\1\grpc\src\core\ext\transport\chttp2\server\chttp2_server.cc","file_line":394,"referenced_errors":[{"created":"@1610658409.777000000","description":"Failed to add port to server","file":"C:\.conan\9877f9d9b72\1\grpc\src\core\lib\iomgr\tcp_server_windows.cc","file_line":510,"referenced_errors":[{"created":"@1610658409.777000000","description":"OS Error","file":"C:\.conan\9877f9d9b72\1\grpc\src\core\lib\iomgr\tcp_server_windows.cc","file_line":201,"os_error":"Only one usage of each socket address (protocol/network address/port) is normally permitted.\r\n","syscall":"bind","wsa_error":10048}]}]}

E0114 22:06:50.866000000  6432 server_chttp2.cc:40] {"created":"@1610658410.866000000","description":"No address added out of total 1 resolved","file":"C:\.conan\9877f9d9b72\1\grpc\src\core\ext\transport\chttp2\server\chttp2_

2nd step : load the hdf5 plugin

In [3]:
base = dpf.BaseService()
base.load_library("Ans.Dpf.Hdf5.dll","hdf5")

#### 3rd step: read stresses, displacements, and the mesh from an rst file and export it to hdf5

In [4]:
stress = model.results.stress()
displacement = model.results.displacement()
mesh = model.metadata.meshed_region

In [5]:
timeIds = list(range(1,model.metadata.time_freq_support.n_sets))

#### By default, we only read the last time step, a user can request one or several time steps using the time_scoping input. Here we will request all time steps

In [6]:
stress.inputs.time_scoping.connect(timeIds)
displacement.inputs.time_scoping.connect(timeIds)

In [7]:
fields = displacement.outputs.fields_container()

In [8]:
field = fields[0]
field.location

'Nodal'

In [9]:
h5op = model.operator("serialize_to_hdf5")

In [10]:
#print(h5op)

#### export to hdf5 with simple precision

In [11]:
h5op.inputs.file_path.connect('c:/temp/dpf_float.h5')

In [12]:
h5op.inputs.data1.connect(stress.outputs)
h5op.inputs.data2.connect(displacement.outputs)
h5op.inputs.data3.connect(mesh)

In [13]:
h5op.run()

#### export to hdf5 with double precision

In [14]:
h5op.inputs.export_floats.connect(False)

In [15]:
h5op.inputs.file_path.connect('c:/temp/dpf_double.h5')

In [16]:
h5op.run()